## Bibliotecas necessárias

Serão necessárias as seguintes bibliotecas

In [62]:
# Garante que as bibliotecas estao instaladas
!pip install geopandas==0.14.4 -q
!pip install folium==0.16.0 -q
!pip install pandas==2.2.2 -q
!pip install numpy==1.26.4 -q
!pip install osmnx==1.9.3 -q
!pip install networkx==3.3 -q

# Importa as bibliotecas
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
import osmnx as ox
import networkx as nx

O Python utilizado está na versão

In [3]:
!python --version

Python 3.12.2


### Bases cartográficas contínuas do Brasil

O Instituto Brasileire do Geografia e Estatística (IBGE) disponibiliza publicamente dados de georeferenciamento do território brasileiro como um todo.

Seus dados são de boa qualidade e disponíveis publicamente em um arquivo "geopackage", que contém todas as camadas de georeferenciamento disponibilizadas.

Como o arquivo pesa ~0.8GB, não faz sentido deixá-lo disponível junto a este notebook. Porém, é possível realizar seu download pelo [site do IBGE](https://www.ibge.gov.br/geociencias/downloads-geociencias.html?caminho=cartas_e_mapas/bases_cartograficas_continuas/bc250/versao2023/). 

O código abaixo utiliza a camada "lml_capital_p", que contém os polígonos dos estados brasileiras.

In [4]:

# Arquivo do IBGE
filename: str = "bc250_2023_11_23.gpkg"

# Limites estaduais
layername: str = "lml_capital_p"

# Le como geoPandas
capitais = gpd.read_file(filename, layer = layername)

# Resume dados do geoPandas
capitais.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   geometriaaproximada  27 non-null     object  
 1   nome                 27 non-null     object  
 2   geocodigo            27 non-null     object  
 3   tipocapital          27 non-null     object  
 4   geometry             27 non-null     geometry
dtypes: geometry(1), object(4)
memory usage: 1.2+ KB


In [107]:
df = pd.DataFrame(capitais[['nome', 'geometry']]).copy()
df[['long', 'lat']] = df['geometry'].astype(str).str.replace('POINT (', '').str.replace(')', '').str.split(' ', expand=True)
df.drop('geometry', axis=1, inplace=True)

df['lat'] = df['lat'].astype(float)
df['long'] = df['long'].astype(float)


df.sort_values(['lat', 'long'], ascending=True, inplace=True)
df

,nome,long,lat
14,Porto Alegre,-51.227035,-30.037858
15,Florianópolis,-48.540472,-27.599537
9,Curitiba,-49.264757,-25.459597
8,São Paulo,-46.648145,-23.564632
4,Rio de Janeiro,-43.194725,-22.912469
7,Campo Grande,-54.618540,-20.468554
26,Vitória,-40.311096,-20.315487
1,Belo Horizonte,-43.951401,-19.891053
23,Goiânia,-49.263088,-16.697998
24,Brasília,-47.880612,-15.783690


## Geração do gráfico

Com os dados coletados, basta utilizar as funções da biblioteca folium, como no exemplo abaixo.

In [126]:
contador: int = 0
ponteiro: int = 0
utilizados: list = []
maximo: int = 1 #len(df)

while contador < maximo:

    item = df.iloc[ponteiro]
    utilizados.append(item['nome'])

    if (ponteiro + 1) == maximo: ponteiro -= maximo
    prox = df.iloc[ponteiro + 1:].copy()
    
    # Pega o ponto mais proximo
    prox['dist'] = (prox['lat'] - item['lat']) + (prox['long'] - item['long'])
    prox = prox[(~prox.nome.isin(utilizados))]
    prox = prox[(prox.dist == min(prox.dist))]

    print(f'{item['nome']} -> {prox['nome']}')  
    
    ponteiro += 1
    contador += 1


Porto Alegre -> 2    Rio Branco
Name: nome, dtype: object


In [139]:
# Convert geoPandas to text json
geojson = capitais.to_json()

# Cria um mapa
map = folium.Map(
    tiles="Cartodb positron",
    location=[-15.7801, -47.9292], # Brasilia 
    zoom_start=4.3, 
    control_scale=True, 
    zoom_control=False,
    max_bounds=True, 
    min_lat=-15,
    max_lat=-15,
    min_lon=-47,
    max_lon=-47
    )

contador: int = -1
ponteiro: int = -1
maximo: int = len(df)
utilizados: list= 


while contador < maximo:

    ponteiro += 1
    contador += 1

    item = df.iloc[ponteiro]
    utilizados.append(item['nome'])
    
    if (ponteiro + 1) == maximo: ponteiro -= maximo
    prox = df.iloc[ponteiro + 1:].copy()
    
    # Pega o ponto mais proximo
    prox['dist'] = (prox['lat'] - item['lat']) + (prox['long'] - item['long'])
    prox = prox[(~prox.nome.isin(utilizados))]
    prox = prox[(prox.dist == min(prox.dist))]

    capital = [item['lat'], item['long']]
    folium.Marker(location=capital).add_to(map)

    destino = [prox['lat'], prox['long']]
    folium.PolyLine([capital, destino]).add_to(map)


# Mostra resultados no mapa
map


/home/miguelzanchettin/anaconda3/envs/mestrado/lib/python3.12/site-packages/folium/utilities.py:94: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/home/miguelzanchettin/anaconda3/envs/mestrado/lib/python3.12/site-packages/folium/utilities.py:100: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/home/miguelzanchettin/anaconda3/envs/mestrado/lib/python3.12/site-packages/folium/utilities.py:102: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]
/home/miguelzanchettin/anaconda3/envs/mestrado/lib/python3.12/site-packages/folium/utilities.py:94: FutureWarning: Calling float on a single element Series is deprecated and will raise a 

ValueError: min() iterable argument is empty